Conversion de PDF a JPEG

In [1]:
# import module
from pdf2image import convert_from_path
import cv2
import numpy as np


# Store Pdf with convert_from_path function
images = convert_from_path('cc1.pdf',500,poppler_path=r'C:\Program Files\poppler-22.11.0\Library\bin')

for i in range(len(images)):
	# Save pages as images in the pdf
	images[i].save('page'+ str(i) +'.jpg', 'JPEG')

PROCESAMIENTO DE IMÁGENES

In [3]:
# LECTURA, REDIMENSIONAMIENTO Y FILTRADO DE IMAGENES
# DOCUMENTO
kernel = np.ones((5,5), np.uint8)
img = cv2.imread('page0.JPG')
img = cv2.resize(img,(800,750))
img_gr = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img_can = cv2.Canny(img,200,200)
# PLANTILLAS
# FRONTAL
pl_frontal = cv2.imread('plantilla_frontal.JPG')
pl_frontal = cv2.resize(pl_frontal,(500,300))
pl_frontal_gr = cv2.cvtColor(pl_frontal,cv2.COLOR_BGR2GRAY)
pl_frontal_can = cv2.Canny(pl_frontal,200,200)
# TRASERA
pl_trasera = cv2.imread('plantilla_trasera.JPG')
pl_trasera = cv2.resize(pl_trasera,(600,300))
pl_trasera_gr = cv2.cvtColor(pl_trasera,cv2.COLOR_BGR2GRAY)
pl_trasera_can = cv2.Canny(pl_trasera,200,200)

# MASCARA
img_dil = cv2.dilate(img_can,kernel,iterations=1)
cv2.imshow('DILATADO', img_dil)
# mascara = cv2.adaptiveThreshold(img_dil, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,95,4)


mascara = cv2.adaptiveThreshold(img_gr, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,13,35)
# CONTORNOS
contornos, _ = cv2.findContours(img_dil,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
for cont in contornos:
    rectangulo = cv2.minAreaRect(cont)
    (x,y), (an, al), angulo = rectangulo
    rect = cv2.boxPoints(rectangulo)
    rect = np.int0(rect)
    cv2.polylines(img,[rect], True,(0,255,0), 2)

cv2.imshow('MASCARA', mascara)

mascara2 = cv2.adaptiveThreshold(pl_frontal_gr, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,13,12)
# cv2.imshow('MASCARA2', mascara2)

# COMPARACIÓN DE IMAGENES
# resultado = cv2.matchTemplate(img_can,pl_trasera_can,cv2.TM_CCOEFF_NORMED)
resultado = cv2.matchTemplate(mascara,mascara2,cv2.TM_CCOEFF_NORMED)
# print(resultado)
min, max, pos_min, pos_max =cv2.minMaxLoc(resultado)

pix_sup_izq = pos_max
pix_inf_der = (pos_max[0]+500, pos_min[1]+50)
print(max,min )

cv2.rectangle(img, pix_sup_izq, pix_inf_der,255 ,4 )
cv2.imshow('RESULTADO',img)
# cv2.imshow('PLANTILLA',pl_trasera_can)

# cv2.imshow('ORIGINAL', img)
# cv2.imshow('CANNY', img_can)
# cv2.imshow('GRISES',img_gr)
# cv2.imshow('ORIGINAL F', pl_frontal)
# cv2.imshow('CANNY F', pl_frontal_can)
# cv2.imshow('GRISES F',pl_frontal_gr)
# cv2.imshow('ORIGINAL T', pl_trasera)
# cv2.imshow('CANNY T', pl_trasera_can)
# cv2.imshow('GRISES T',pl_trasera_gr)
cv2.waitKey(0)


0.11023230850696564 -0.05231846496462822


-1